# 使用 PyTorch 完成 logistic 回归

上一节我们介绍了简单的线性回归，如何在 pytorch 里面用最小二乘法来拟合一些离散的点，这一节我们将开始简单的 logistic 回归，介绍图像分类问题，使用的数据是手写字体数据集 MNIST 。

## 1、logistic 回归
logistic 回归简单来说和线性回归是一样的，要做的运算同样是 y = w * x + b ，logistic 回归简单的是做二分类问题的，使用 sigmoid 函数将所有的正数和负数都变成 0-1 之间的数，这样就可以用这个数来确定到底属于哪一类，可以简单的认为概率大于 0.5 即为第二类，小于 0.5 为第一类。

![sigmoid函数](img/pt_lo_1.png)

而我们这里要做的是多分类的问题，对于每一个数据，我们输出的维数是分类的总数，比如 10 分类，我们输出的就是一个 10 维的向量，然后我们使用另外一个激活函数，softmax ，如下：

![softmax](img/pt_lo_2.png)

这就是 softmax 函数作用的机制，其实简单的理解就是确定这 10 个数每个数对应的概率有多大，因为这 10 个数有正有负，所以通过指数函数将他们全部变成正数，然后求和，然后这 10 个数每个数都除以这个和，这样就得到了每个类别的概率。

## 2、使用 logistic 回归

首先我们要导入 torch 里面专门做图形处理的一个库，torchvision ，根据官方安装指南，你在安装 pytorch 的时候 torchvision 也会安装。

我们需要使用的是 torchvision.transforms 和 torchvision.datasets 以及 torch.utils.data.DataLoader 

首先 DataLoader 是导入图片的操作，里面有一些参数，比如 batch_size 和 shuffle 等，默认 load 进去的图片类型是 PIL.Image.open 的类型，如果你不知道 PIL ，简单来说就是一种读取图片的库。

torchvision.transforms 里面的操作是对导入的图片做处理，比如可以随机取 (50, 50) 这样的窗框大小，或者随机翻转，或者去中间的 (50, 50) 的窗框大小部分等等，但是里面必须要用的是 transforms.ToTensor() ，这可以将 PIL 的图片类型转换成 tensor ，这样 pytorch 才可以对其做处理

torchvision.datasets 里面有很多数据类型，里面有官网处理好的数据，比如我们要使用 MNIST 数据集，可以使用 torchvision.datasets.MNIST() 来得到，还有一个常使用的是 torchvision.datasets.ImageFolder() ，这个可以让我们按文件夹来取图片，和 keras 里面的 flow_from_directory() 类似，具体的可以去看看官方文档的介绍。

### 2.1、引入相应的库

In [2]:
# 引入相应的库
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils import data
from torch.autograd import Variable
import torchvision.transforms as transforms

### 2.2、下载训练数据集 MNIST 手写数字数据集

In [3]:
# 下载得到 MNIST 手写数字数据集
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=False)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

In [4]:
# 定义超参数
batch_size = 32
learning_rate = 1e-3
num_epoches = 10

# 设置好加载器
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

以上就是我们对图片数据的读取操作。

### 2.3、构建模型
我们之前讲过模型定义的框架，废话不多说，我们直接上代码

In [5]:
class Logistic_Regression(nn.Module):
    # 定义初始化函数
    def __init__(self, in_dim, n_class):
        super(Logistic_Regression, self).__init__()
        # 我们这里的 logistic 使用的是 Linear，也就是 y = kx + b 这种形式
        self.logistic = nn.Linear(in_dim, n_class)
    # 定义前向传播    
    def forward(self, x):
        out = self.logistic(x)
        return out
    
model = Logistic_Regression(28*28, 10) # 图片大小是 28x28

我们需要向这个模型传入参数，第一个参数定义为数据的维度，第二维度是我们分类的数目。

接着我们可以在 gpu 上跑模型，但是呢？我们没有 gpu ，想想就心累。。。

首先我们判断一下你是否能在 gpu 上跑，如下：

In [6]:
torch.cuda.is_available() # 可以在 gpu 上跑的话，就是 True，不可以的话，就是 False

False

如果返回 True 就说明有 gpu 支持。否则，会返回 False ，表示不支持 gpu 。

如果可以的话，你就只需要一个命令就可以运行了。如下：

In [7]:
# 我本机上没有装 gpu ，所以就不运行这部分了，如果没有装 gpu 运行的时间会长一些
# model = model.cuda()
# 或者
# model.cuda()

然后，接下来是我们进行 loss 和 optimizer 的定义

In [8]:
# 我们使用 交叉熵 来做 loss 标准
criterion = nn.CrossEntropyLoss()
# 使用 随机梯度下降 作为
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

这里我们使用的 loss 是交叉熵，是一种处理分类问题的 loss ，optimizer 我们还是使用随机梯度下降。

**本来我们使用的 softmax 函数，在代码中显示写出了，但是有大佬指出如下错误：**

**out = F.softmax(out)这一行是不需要的，nn.CrossEntropyLoss()本身会自动算softmax，你这样实际上是算了两次softmax，所以准确率会降低一些**

所以我们就将原代码中的 softmax() 去掉了。

### 2.3、训练模型
接下来我们就开始训练了。

In [11]:
# 判断是否使用 gpu
use_gpu = torch.cuda.is_available()

for epoch in range(num_epoches):
    print('epoch {}'.format(epoch+1))
    print('*'*10)
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        img = img.view(img.size(0), -1)  # 将图片展开成 28x28
        if use_gpu:
            img = Variable(img).cuda()
            label = Variable(label).cuda()
        else:
            img = Variable(img)
            label = Variable(label)
        # 向前传播
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss.data[0] * label.size(0)
        # torch.max(a,1) 返回每一行中最大值的那个元素，且返回其索引（返回最大元素在这一行的列索引）
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        running_acc += num_correct.data[0]
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Loss --", running_loss)
    print("Acc --", running_acc)

epoch 1
**********


D:\anacondapy3.6\lib\site-packages\ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
D:\anacondapy3.6\lib\site-packages\ipykernel_launcher.py:24: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Loss -- tensor(24990.2344)
Acc -- tensor(53361)
epoch 2
**********
Loss -- tensor(24728.8574)
Acc -- tensor(53422)
epoch 3
**********
Loss -- tensor(24485.3789)
Acc -- tensor(53456)
epoch 4
**********
Loss -- tensor(24257.7070)
Acc -- tensor(53506)
epoch 5
**********
Loss -- tensor(24046.9590)
Acc -- tensor(53558)
epoch 6
**********
Loss -- tensor(23848.0723)
Acc -- tensor(53600)
epoch 7
**********
Loss -- tensor(23660.5352)
Acc -- tensor(53632)
epoch 8
**********
Loss -- tensor(23483.3887)
Acc -- tensor(53682)
epoch 9
**********
Loss -- tensor(23316.9961)
Acc -- tensor(53715)
epoch 10
**********
Loss -- tensor(23159.0293)
Acc -- tensor(53742)


然后我们可以测试模型，过程与训练类似，只是注意要将模型改成测试模式。

```python
model.eval()
```

具体的结果，多久打印一次，以及如何打印都可以自己在 for 循环里设计。

## 3、小结
这一部分，我们就讲解了如何用 logistic 回归做一个简单的图片分类问题，知道了如何在 gpu 上跑模型，下一节我们介绍如何写简单的卷积神经网络。不了解卷积神经网络的同学可以提前先去了解一下。